# Part 3: GLMM Analysis


## 1. Creating the model predictors using GridFix

In this part of the tutorial, we will use the grid and image features defined in the previous chapters to create a GLMM predictor matrix and output some model code for R. First, let's reproduce the analysis from previous chapters and add some features that might influence each grid cell's fixation probability:

In [1]:
%matplotlib inline

import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

from gridfix import *

# Load images and define 8x6 grid from part 1
images = ImageSet('images/tutorial_images.csv', label='tutorial')
grid = GridRegionSet(size=images.size, gridsize=(8,6), label='testgrid')

# Define some simple features from part 2
fLum = LuminanceFeature(grid, images)
fEdge = SobelEdgeFeature(grid, images)
fCent = CentralBiasFeature(grid, images, measure='gaussian', sig2=0.23, nu=0.45)

# Load IKN98 feature maps and define a MapFeature
ids = ['112', '67', '6', '52', '37', '106', '129', '9', '107', '97', '58', '111', '85', '149', '150']
maps = ImageSet('maps', imageids=ids, mat_var='IKN98')
fIKN = MapFeature(grid, maps, stat=np.mean)

# Load fixation data
fix = Fixations('tutorial_fixations.csv', imageid='image_id', fixid='CURRENT_FIX_INDEX', 
                x='CURRENT_FIX_X', y='CURRENT_FIX_Y', imageset=images)

Now the actual GLMM preprocessing can be performed using a _FixationModel_ object which combines fixation data, RegionSet and all features specified in the _features=_ argument into a common predictor matrix, which can then be loaded into R. With a big dataset, this could take a while, but for this tutorial, updating the model should be a matter of a few seconds. Note that the _chunks=_ argument specifies the names of columns over which data should not be aggregated, e.g. individual subjects, while the _features=_ argument contains a Python list of the actual Feature objects defined above.

In our example, this predictor matrix contains one line for each subject, image and grid cell, which will yield 8 x 15 x 48 = 5760 individual data points to be entered into GLMM. We can print the resulting model object to verify this:

In [2]:
model = FixationModel(fix, grid, chunks=['subject_number', 'image_id'], features=[fLum, fCent], dv_type='fixated')
print(model)

<gridfix.FixationModel, 5760 samples, DV=fixated, chunked by: ['subject_number', 'image_id']>
Fixations:
	<gridfix.Fixations data set, 2556 samples, 15 images>
Images:
	<gridfix.ImageSet "tutorial", 15 images, size=(800, 600), normalized>
Regions:
	<gridfix.GridRegionSet (testgrid), size=(800, 600), 8x6 grid, 48 cells, memory=22500.0 kB>

Features:
	fCentr	CentralBiasFeature
	fLumin	LuminanceFeature



The resulting predictor matrix now contains one row per combination of image and region (and possible other variables used for chunking in the model specification, such as the subject id). Within each row, the column _dvFix_ indicates the fixation state of each cell, i.e., whether it was fixated (1) or not (0), while the remaining columns contain the feature values for the corresponding regions - here, mean cell luminance (fLumin) and distance from image center following an anisotropic Gaussian CB model (fCentr). 

The predictor matrix can be accessed as a DataFrame using the _predictors_ attribute of the generated model object:

In [3]:
print(model.predictors)

    subject_number image_id  region  dvFix    fCentr    fLumin
0            201.0      106     1.0    0.0  0.971265  0.484337
1            201.0      106     2.0    0.0  0.935023  0.484956
2            201.0      106     3.0    0.0  0.888111  0.567180
3            201.0      106     4.0    0.0  0.853275  0.657081
4            201.0      106     5.0    0.0  0.853474  0.792208
5            201.0      106     6.0    0.0  0.888567  0.800865
6            201.0      106     7.0    0.0  0.935464  0.779572
7            201.0      106     8.0    0.0  0.971537  0.681514
8            201.0      106     9.0    0.0  0.903758  0.489519
9            201.0      106    10.0    1.0  0.782377  0.395110
10           201.0      106    11.0    0.0  0.625257  0.555128
11           201.0      106    12.0    0.0  0.508580  0.538794
12           201.0      106    13.0    0.0  0.509249  0.882058
13           201.0      106    14.0    0.0  0.626785  0.983519
14           201.0      106    15.0    0.0  0.783854  0

To facilitate analysis of the generated predictor matrix in R, GridFix also generates R source code that contains the necessary commands to import the generated data file, define factors, standardize predictors and set up a model formula. This can serve as a starting point for analysis but should be adapted to the individual factor structure of each hypothesis and dataset - as a reminder, the actual call to _glmer_ is commented out. 

The _model.save()_ function saves both the predictor matrix and the R code to files to be read into R. Note that the file name argument should be the _base name_ of generated files, e.g., model.save("tutorial") will generate tutorial.csv and tutorial.R.

In [4]:
# Print the source code here as an example
print(model.r_source())

model.save('tutorial')

# GridFix GLMM R source, generated on 08.05.17, 14:27:17
# input file:	gridfix.csv
# RegionSet:	<gridfix.GridRegionSet (testgrid), size=(800, 600), 8x6 grid, 48 cells, memory=22500.0 kB>
# DV type:	fixated

library(lme4)

data  <- read.table("gridfix.csv", header=T, sep="\t", row.names=NULL)

# Define R factors for all chunking variables and group dummy codes
data$subject_number <- as.factor(data$subject_number)
data$image_id <- as.factor(data$image_id)

# Center and scale predictors
data$fCentr_C <- scale(data$fCentr, center=TRUE, scale=TRUE)
data$fLumin_C <- scale(data$fLumin, center=TRUE, scale=TRUE)

# NOTE: this source code can only serve as a scaffolding for your own analysis!
# You MUST adapt the GLMM model formula below to your model, then uncomment the corresponding line!
#model <- glmer(dvFix ~ 1 + fCentr_C  + fLumin_C  + (1 | image_id), data=data, family=binomial)

save(file="gridfix_GLMM.Rdata", list = c("model"))

print(summary(model))



## 2. Evaluating the model using R

**Note: The following cells will only work interactively if you have a working R environment and the rpy2 Python module installed. The next cell will set up the %%R magic code to run R code within this notebook**

In [5]:
# Initialize R environment for Jupyter notebook using rpy2
%load_ext rpy2.ipython

ImportError: No module named 'rpy2'

The Python code cell above saved the tutorial model (central bias and luminance) to "tutorial.csv", which we will now load into R. The following code example fits a model containing central bias and mean cell luminance as fixed factors and includes random slopes (but not random intercepts) for individual images. The resulting R model object can then be manipulated using R commands as usual and/or saved to an Rdata file.

In [ ]:
%%R
# GridFix GLMM R source, generated on 04.05.17, 14:22:46
# input file:	gridfix.csv
# RegionSet:	<gridfix.GridRegionSet (testgrid), size=(800, 600), 8x6 grid, 48 cells, memory=22500.0 kB>
# DV type:	fixated

library(lme4)

data  <- read.table("tutorial.csv", header=T, sep="\t", row.names=NULL)

# Define R factors for all chunking variables and group dummy codes
data$subject_number <- as.factor(data$subject_number)
data$image_id <- as.factor(data$image_id)

# Center and scale predictors
data$fCentr_C <- scale(data$fCentr, center=TRUE, scale=TRUE)
data$fLumin_C <- scale(data$fLumin, center=TRUE, scale=TRUE)

# NOTE: this source code can only serve as a scaffolding for your own analysis!
# You MUST adapt the GLMM model formula below to your model, then uncomment the corresponding line!
model <- glmer(dvFix ~ 1 + fCentr_C  + fLumin_C  + (1 | image_id), control=glmerControl(optimizer="bobyqa"), data=data, family=binomial)

print(summary(model))

## 4. Concluding Remarks

This concludes the GridFix tutorial - we hope that it can prove helpful in setting up a preprocessing script and GLMM-based fixation analysis. For more details of supported image features and other attributes of the GridFix toolbox, you can use the navigation bar on the left to browse the module documentation or look at some example scripts for common analyses. Thank you for your interest in this method!
